In [ ]:
# automatically reload modules if source is modified 
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pathlib
import enum 

In [ ]:
# `easyvvuq` dependencies
import easyvvuq as vvuq
import chaospy as cp

In [ ]:
# `isct` dependencies 
from workflow.uq import ISCTEncoder, ISCTDecoder
from workflow.isct_trial import trial as trial_cmd 

User directories: 
- `template_dir`: this directory points towards a patient directory that acts as template, i.e. this directory (and all its content) are duplicated for each new run generated by `EasyVVUQ`. Therefore, it is suggested to perform one analysis of that patient directory before generating the new directories with `EasyVVUQ`, as all initialisations and preprocessing is simply copied, rather then needed to be repeated. For example, create and run a single patient:
```
isct trial create one -n 1 
isct trial run one -v 
```

- `work_dir`: this is the directory in which `EasyVVUQ` will create the runs and corresponding databases. Everytime `EasyVVUQ` is started it generates a new database inside this folder (with a rather obscure random hash attached to the directory name). Be sure to clear out this `work_dir` somewhat regularly, as many directories can accumulate taking up some diskspace. 

In [ ]:
template_dir = "/Users/max/trials/one/patient_000"
work_dir = "/Users/max/trials/vvuq"

In [ ]:
# ensure input is present
template_dir = pathlib.Path(template_dir)
assert os.path.isdir(template_dir)

work_dir = pathlib.Path(work_dir)
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
assert os.path.isdir(work_dir) 

### Campaing definition 

In [ ]:
# create a `campaign`
campaign = vvuq.Campaign('UQ_', work_dir=work_dir)

In [ ]:
# create a `encoder` to encode the parameters towards input files
encoder = ISCTEncoder(template_fname=template_dir)

In [ ]:
# output file where `pressure drop` data is written to 
output = "bf_sim/Pressure_Drop.csv"

# the columns of variables to extract
cols = ["pressure_drop", "BLOOD_VISC"]

# create a `decoder` to decode the output parameters
decoder = ISCTDecoder(target_filename=output, output_columns=cols)

In [ ]:
# create a `collater` to aggregate the output data 
collater = vvuq.collate.AggregateSamples(average=False)

In [ ]:
# define parameters of interest and their properties
# this all just goes into a single dictionary, where now only 
# `BLOOD_VISC` is considered as parameter to be varied 
parameters = {
    "BLOOD_VISC": {
        "type": "float", 
        "min": 0.025, 
        "max": 0.055, 
        "default": 0.035,
    }
}

In [ ]:
# create an `app` for the campaign, by connecting all components
campaign.add_app(
    name="blood-visc",
    params=parameters,
    encoder=encoder,
    decoder=decoder,
    collater=collater,
)

### Sampling definition

In [ ]:
# the parameters to vary are provided as dict with their 
# corresponding distributions 
vary = {
    "BLOOD_VISC": cp.Uniform(0.025, 0.055),
}

In [ ]:
# available methods
class Method(enum.Enum):
    random = "random"
    PCE = "PCE"

In [ ]:
# pick any from the Enum 
method = Method.PCE 

# create a `sampler` matching the method 
if method == method.random: 
    sampler = vvuq.sampling.RandomSampler(vary=vary)
    
if method == method.PCE: 
    sampler = vvuq.sampling.PCESampler(vary=vary, polynomial_order=3)

# assign the sampler
campaign.set_sampler(sampler)

In [ ]:
# draw the samples
num_samples = 5
replicas = 1 # the number of times a single sample is replicated
campaign.draw_samples(num_samples=num_samples, replicas=replicas)

In [ ]:
# log the `BLOOD_VISC` parameter for each run proposed by the sampler
for run in campaign.list_runs():
    print(f"{run[1]['run_name']}: {run[1]['params']}")

In [ ]:
# create all run directories; copies the template and updates the
# parameters using the `ISCTEncoder`
campaign.populate_runs_dir()

### Running `isct` for each proposed sample

Note: the cells below either evaluate all simulations, or provide you a command to start them in the terminal locally, however, this step can also be skipped if you just want to test the remainder of the output. In that case, all analyses are performed on the copied data. Thus, nothing statistic meaningful is found, but it does allow to make sure the analysis doesn't simply crash. 

In [ ]:
# extract the path from the database location, this seems required
# to obtain the hash that is attached after the original directory 
run_dir = campaign.db_location.split(":")[-1]
run_dir = pathlib.Path(run_dir).parent

# the runs are located in the /runs/ directory
run_dir = run_dir.joinpath("runs")

In [ ]:
# issue `isct` to perform the simulation on all runs 
# NOTE: the output seems to be hidded by the notebook, i.e. it is
# logged in the terminal console, rather than in the notebook 

# either run this command
# trial_cmd(f"trial run {run_dir} -v".split())
# or run the following output in terminal 
print(f"isct trial run {run_dir} -v")

### Collecting output

In [ ]:
campaign.collate()

### Analysing output

In [ ]:
# define analysis in line with sampling method
if method == method.random:
    analysis = vvuq.analysis.BasicStats(
        qoi_cols=["BLOOD_VISC", "pressure_drop"]
    )

if method == method.PCE: 
    analysis = vvuq.analysis.PCEAnalysis(
        sampler=sampler, 
        qoi_cols=["BLOOD_VISC", "pressure_drop"]
    )
    
# apply analysis to current database
campaign.apply_analysis(analysis)
results = campaign.get_last_analysis()

In [ ]:
results